# Databricks Notebooks: Data Processing with Python and SQL

**Scenario:** You've found the aircraft IoT sensor data. Now leadership needs you to analyze it, find patterns, and prepare it for dashboards and ML models. All before end of week!

**Databricks Notebooks** are interactive documents where you'll write Python and SQL to process data.

## Pre-Read: Python in 60 Seconds

**New to Python?** Here's all you need to get started:

### Variables
Variables store data. No declaration needed:
```python
temperature = 75.5          # Number
factory_name = "Boeing"     # Text (string)
is_critical = True          # Boolean (True/False)
device_ids = [1, 2, 3, 4]   # List
```

### Functions
Functions are reusable blocks of code:
```python
def calculate_average(numbers):
    total = sum(numbers)
    count = len(numbers)
    return total / count

result = calculate_average([10, 20, 30])  # result = 20.0
```

**That's it!** You now know enough Python for this notebook. We'll explain everything else as we go.

---

## What You'll Learn

✅ Read data from Unity Catalog  
✅ Transform data with PySpark  
✅ Calculate aggregations  
✅ Create visualizations  
✅ Save processed data  

**Time to Complete:** 25-30 minutes

---

## Table of Contents

1. Notebook Basics
2. Reading Data
3. DataFrame Operations
4. Joining Tables
5. Visualizations

---

**References:**
- [Notebooks Documentation](https://docs.databricks.com/aws/en/notebooks/)
- [Notebooks Code](https://docs.databricks.com/aws/en/notebooks/notebooks-code)


In [0]:
# Configuration - UPDATE THESE VALUES!
CATALOG = 'your_catalog'  # Update: Change to your catalog name
SCHEMA = 'your_username'  # Update: Use your username (without special characters)

print(f"✅ Using catalog: {CATALOG}")
print(f"✅ Using schema: {SCHEMA}")
print(f"📊 Tables will be at: {CATALOG}.{SCHEMA}.table_name")


## 1. Notebook Basics

### What are Databricks Notebooks?

Notebooks are interactive documents containing:
- **Code cells** - Execute Python, SQL, Scala, or R code
- **Markdown cells** - Documentation and explanations
- **Visualizations** - Built-in plotting capabilities
- **Results** - Output from code execution

### Key Features:

✅ **Multi-language support** - Switch between languages in the same notebook  
✅ **Collaboration** - Real-time co-editing with teammates  
✅ **Version control** - Git integration for tracking changes  
✅ **Scheduling** - Run notebooks as automated jobs  
✅ **Interactive visualizations** - Built-in charting  

### Magic Commands:

- `%python` - Python code (default)
- `%sql` - SQL queries
- `%scala` - Scala code
- `%r` - R code
- `%md` - Markdown for documentation
- `%sh` - Shell commands
- `%pip` - Install Python packages

### Keyboard Shortcut:

- `Shift + Enter` - Run cell and move to next

## 2. Reading Data

### Reading from Unity Catalog Tables

The simplest way to read data is using `spark.table()`:


In [0]:
# Read a table from Unity Catalog
sensors_df = spark.table(f"{CATALOG}.{SCHEMA}.sensor_bronze")

# Show schema
print("Schema:")
sensors_df.printSchema()

# Display first few rows
sensors_df.limit(5).display()


### Reading from Files in Volumes


In [0]:
# Read CSV files from a volume
csv_path = f"/Volumes/{CATALOG}/{SCHEMA}/sensor_data/"

df_from_volume = (
    spark.read
    .format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(csv_path)
)

print(f"Records read from volume: {df_from_volume.count():,}")
df_from_volume.limit(3).display()


### Basic DataFrame Info


In [0]:
# Get DataFrame information
print(f"Total rows: {sensors_df.count():,}")
print(f"Total columns: {len(sensors_df.columns)}")
print(f"\nColumns: {sensors_df.columns}")

# Show summary statistics
sensors_df.select("temperature", "rotation_speed", "air_pressure").summary().display()


## 3. DataFrame Operations

### Selecting Columns


In [0]:
from pyspark.sql.functions import col

# Select specific columns
selected_df = sensors_df.select(
    "device_id",
    "timestamp",
    "temperature",
    "rotation_speed",
    "factory_id"
)

selected_df.limit(5).display()


### Filtering Data


In [0]:
# Filter for high temperatures
high_temp_df = sensors_df.filter(col("temperature") > 80)

print(f"High temperature readings: {high_temp_df.count():,}")
high_temp_df.limit(5).display()

# Multiple conditions
critical_df = sensors_df.filter(
    (col("temperature") > 80) & 
    (col("rotation_speed") > 600)
)

print(f"\nCritical readings (high temp AND high speed): {critical_df.count():,}")


### Adding and Transforming Columns


In [0]:
from pyspark.sql.functions import col, when, round as spark_round

# Add temperature in Celsius
transformed_df = sensors_df.withColumn(
    "temperature_celsius",
    spark_round((col("temperature") - 32) * 5/9, 2)
)

# Add a status flag
transformed_df = transformed_df.withColumn(
    "temperature_status",
    when(col("temperature") > 85, "Critical")
    .when(col("temperature") > 75, "Warning")
    .otherwise("Normal")
)

# Calculate derived metric
transformed_df = transformed_df.withColumn(
    "performance_index",
    spark_round(col("rotation_speed") / col("air_pressure") * 100, 2)
)

transformed_df.select(
    "device_id",
    "temperature",
    "temperature_celsius",
    "temperature_status",
    "performance_index"
).limit(10).display()


### Handling Null Values


In [0]:
# Check for nulls
from pyspark.sql.functions import count, when, col, isnan

null_counts = sensors_df.select([
    count(when(col(c).isNull(), c)).alias(c) 
    for c in sensors_df.columns
])

print("Null counts per column:")
null_counts.display()

# Drop rows with any nulls
clean_df = sensors_df.na.drop()

# Fill nulls with specific values
filled_df = sensors_df.fillna({
    "temperature": 0,
    "air_pressure": sensors_df.agg({"air_pressure": "mean"}).first()[0]
})

print(f"\nOriginal: {sensors_df.count():,} rows")
print(f"After dropping nulls: {clean_df.count():,} rows")


## 4. Joining Tables

### Inner Join with Dimension Tables


In [0]:
# Load dimension tables
dim_factories = spark.table(f"{CATALOG}.{SCHEMA}.dim_factories")
dim_models = spark.table(f"{CATALOG}.{SCHEMA}.dim_models")

# Join sensor data with factories
enriched_df = (
    sensors_df
    .join(dim_factories, "factory_id", "inner")
    .join(dim_models, "model_id", "inner")
    .select(
        "device_id",
        "timestamp",
        "temperature",
        "rotation_speed",
        "factory_name",
        "region",
        "city",
        "model_name",
        "model_category"
    )
)

print("Enriched sensor data:")
enriched_df.limit(10).display()


### Calculate Aggregations on Enriched Data

Now that we have enriched data with factory and model names, let's calculate metrics:


In [0]:
from pyspark.sql.functions import avg, count, round as spark_round

# Performance by region and model category
region_model_stats = (
    enriched_df
    .groupBy("region", "model_category")
    .agg(
        count("*").alias("reading_count"),
        spark_round(avg("temperature"), 2).alias("avg_temperature"),
        spark_round(avg("rotation_speed"), 2).alias("avg_rotation_speed")
    )
    .orderBy("region", col("avg_temperature").desc())
)

print("Performance by Region and Model Category:")
region_model_stats.display()


## 5. Visualizations

### Built-in Display Visualizations

Databricks notebooks have built-in visualization capabilities. Let's create some charts:


In [0]:
# Prepare data for visualization
factory_temp_viz = (
    enriched_df
    .groupBy("factory_name")
    .agg(
        spark_round(avg("temperature"), 2).alias("avg_temperature"),
        spark_round(avg("rotation_speed"), 2).alias("avg_rotation_speed")
    )
    .orderBy(col("avg_temperature").desc())
)

# Display - click the chart icon to create visualizations
factory_temp_viz.display()

print("""
📊 Try These Visualizations:
1. Click the bar chart icon below the table
2. Drag 'factory_name' to Keys
3. Drag 'avg_temperature' to Values
4. Try different chart types: bar, line, pie, scatter
""")


## Summary

Congratulations! You've learned the basics of data processing with Databricks Notebooks.

### What You Accomplished

✅ **Understood Python basics** - Variables and functions  
✅ **Read data from Unity Catalog** - Loaded sensor tables  
✅ **Transformed data** - Filtered, selected, and added columns  
✅ **Joined tables** - Combined sensor data with dimension tables  
✅ **Created aggregations** - Calculated metrics by region and model  
✅ **Visualized data** - Built charts in the notebook  

### Key Takeaways

1. **DataFrames are immutable** - Each transformation returns a new DataFrame
2. **PySpark is like SQL** - Similar operations (select, filter, groupBy, join)
3. **display()** shows results with automatic visualizations
4. **Joins enrich data** - Combine fact tables with dimensions for context
5. **Aggregations summarize** - groupBy + agg to calculate metrics

### What's Next

Now you can:
- Build more complex transformations (Day 1, Notebook 5)
- Create dashboards from your processed data (Day 1, Notebook 3)
- Use this data for ML models (Day 2)

**You're making great progress toward leadership's deadline!** 🚀

---

## Try This Out

Want more practice? Try these exercises using what you learned:

### 1. Use Databricks Assistant to Add a Window Function

**Challenge:** Calculate a 3-row moving average of temperature for each device.

**Steps:**
1. Select a cell with DataFrame code
2. Click the AI Assistant icon (or press Cmd/Ctrl + Shift + Space)
3. Type: "Add a column with 3-row moving average of temperature partitioned by device_id"
4. Review and run the generated code

**Hint:** You'll need `Window.partitionBy("device_id").orderBy("timestamp").rowsBetween(-1, 1)`

### 2. Try Different Aggregations

Practice more aggregation functions:
```python
# Find devices with high temperature variance
from pyspark.sql.functions import stddev, min, max

device_variance = (
    sensors_df
    .groupBy("device_id")
    .agg(
        count("*").alias("reading_count"),
        avg("temperature").alias("avg_temp"),
        stddev("temperature").alias("stddev_temp"),
        min("temperature").alias("min_temp"),
        max("temperature").alias("max_temp")
    )
    .orderBy(col("stddev_temp").desc())
)

device_variance.limit(10).display()
```

**Questions to explore:**
- Which devices have the highest temperature variance?
- Is high variance correlated with defects?
- Do certain factories have more variance?

### 3. Create More Visualizations

Try different chart types:
- **Line chart** - Temperature trends over time by factory
- **Pie chart** - Distribution of readings by region
- **Scatter plot** - Temperature vs. rotation_speed (looking for correlations)
- **Heatmap** - Average temperature by factory and model

### 4. Practice SQL Magic

Did you know you can write SQL directly in notebooks?

Create a new cell and type:
```sql
%sql
SELECT 
    factory_id,
    COUNT(*) as reading_count,
    AVG(temperature) as avg_temp
FROM your_catalog.your_schema.sensor_bronze
WHERE timestamp >= current_date() - 7
GROUP BY factory_id
ORDER BY avg_temp DESC
```

### 5. Explore More Tables

Apply what you learned to other tables:
- `inspection_bronze` - Quality inspection data
- `anomaly_detected` - Flagged anomalies
- `inspection_silver` - Processed inspection data

**Try:**
- Join inspections with sensor data
- Calculate defect rates by factory
- Find correlation between sensor readings and defects

---

**Additional Resources:**
- [Notebooks Documentation](https://docs.databricks.com/aws/en/notebooks/)
- [PySpark API Reference](https://spark.apache.org/docs/latest/api/python/)
- [Databricks Assistant Guide](https://docs.databricks.com/aws/en/notebooks/databricks-assistant)

**Great job!** You now have the skills to process IoT data and create insights for leadership. Keep going! 💪


In [0]:
# Load dimension tables
dim_factories = spark.table(f"{CATALOG}.{SCHEMA}.dim_factories")
dim_models = spark.table(f"{CATALOG}.{SCHEMA}.dim_models")

# Join sensor data with factories
enriched_df = (
    sensors_df
    .join(dim_factories, "factory_id", "inner")
    .join(dim_models, "model_id", "inner")
    .select(
        "device_id",
        "timestamp",
        "temperature",
        "rotation_speed",
        "factory_name",
        "region",
        "city",
        "model_name",
        "model_category"
    )
)

print("Enriched sensor data:")
enriched_df.limit(10).display()


### Aggregate Enriched Data


In [0]:
# Performance by region and model category
region_model_stats = (
    enriched_df
    .groupBy("region", "model_category")
    .agg(
        count("*").alias("reading_count"),
        spark_round(avg("temperature"), 2).alias("avg_temperature"),
        spark_round(avg("rotation_speed"), 2).alias("avg_rotation_speed")
    )
    .orderBy("region", col("avg_temperature").desc())
)

print("Performance by Region and Model Category:")
region_model_stats.display()


## 6. Visualizations

### Built-in Display Visualizations

Databricks notebooks have built-in visualization capabilities:


In [0]:
# Prepare data for visualization
factory_temp_viz = (
    enriched_df
    .groupBy("factory_name")
    .agg(
        spark_round(avg("temperature"), 2).alias("avg_temperature"),
        spark_round(avg("rotation_speed"), 2).alias("avg_rotation_speed")
    )
    .orderBy(col("avg_temperature").desc())
)

# Display - click the chart icon to create visualizations
factory_temp_viz.display()

print("""
📊 Try These Visualizations:
1. Click the bar chart icon below the table
2. Drag 'factory_name' to Keys
3. Drag 'avg_temperature' to Values
4. Try different chart types: bar, line, pie, scatter
""")


## 7. Writing Data

### Save Transformed Data as a New Table


In [0]:
# Create a processed dataset
processed_df = (
    enriched_df
    .withColumn("temperature_celsius", spark_round((col("temperature") - 32) * 5/9, 2))
    .withColumn(
        "temperature_category",
        when(col("temperature") > 85, "Critical")
        .when(col("temperature") > 75, "High")
        .when(col("temperature") > 65, "Normal")
        .otherwise("Low")
    )
    .withColumn("processing_timestamp", current_timestamp())
)

# Save as Delta table
output_table = f"{CATALOG}.{SCHEMA}.sensor_processed"

processed_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(output_table)

print(f"✅ Data saved to: {output_table}")
print(f"   Total records: {processed_df.count():,}")


### Append vs Overwrite Modes


In [0]:
from pyspark.sql.functions import current_timestamp

# Example: Append mode (adds new records)
# processed_df.write.format("delta").mode("append").saveAsTable(output_table)

# Example: Overwrite mode (replaces all data)
# processed_df.write.format("delta").mode("overwrite").saveAsTable(output_table)

# Example: Write to a specific location
# processed_df.write.format("delta").mode("overwrite").save(f"/Volumes/{CATALOG}/{SCHEMA}/processed_data/")

print("""
Write Modes:
- overwrite: Replace existing data
- append: Add new records
- ignore: Skip if table exists
- error: Fail if table exists (default)
""")


## Summary

In this notebook, you learned:

✅ **Notebook basics** - Magic commands, keyboard shortcuts, collaboration  
✅ **Reading data** - From Unity Catalog tables and volumes  
✅ **DataFrame operations** - Select, filter, transform columns  
✅ **Aggregations** - GroupBy, window functions, statistics  
✅ **Joins** - Enrich data with dimension tables  
✅ **Visualizations** - Built-in charts and custom HTML  
✅ **Writing data** - Save transformed data to Delta tables  

### Key Takeaways:

1. **DataFrames are immutable** - Each transformation returns a new DataFrame
2. **Lazy evaluation** - Transformations are planned, not executed until an action
3. **display()** - Best way to view results with automatic visualizations
4. **Chaining operations** - Use parentheses for readable multi-line transformations
5. **Unity Catalog** - Simplifies data access with three-level namespace

### PySpark Best Practices:

**Performance:**
- Use `filter()` early to reduce data volume
- Avoid `collect()` on large datasets (brings all data to driver)
- Use `cache()` for DataFrames accessed multiple times
- Partition output data appropriately

**Code Quality:**
- Use explicit column references with `col()`
- Chain transformations for readability
- Add comments for complex logic
- Use meaningful variable names

**Data Quality:**
- Check for nulls and handle them explicitly
- Validate data types match expectations
- Use `na.drop()` or `fillna()` strategically
- Add data quality checks before writing

### Common DataFrame Actions:

**Transformations (lazy):**
- `select()`, `filter()`, `withColumn()`, `groupBy()`, `join()`

**Actions (trigger execution):**
- `display()`, `show()`, `count()`, `collect()`, `write()`

### Try These:

- Explore **SQL magic** (`%sql`) for SQL queries in notebooks
- Learn about **Delta Lake** features (time travel, MERGE, OPTIMIZE)
- Try **Structured Streaming** for real-time data processing

---

**Additional Resources:**
- [Notebooks Documentation](https://docs.databricks.com/aws/en/notebooks/)
- [PySpark API Reference](https://spark.apache.org/docs/latest/api/python/)
- [Delta Lake Guide](https://docs.databricks.com/aws/en/delta/)
